# Dixit
Here we present simulation and estimation of the model described in \cite{AK}.
The reference is one of the few tutorials out there going through an example from
mathematics to code, so again it seems worthwhile to try and implement this model
using our package.

The model is again an entry/exit model. The state space has two states: the
current incumbency of the firm, and a business cycle state. The business cycle
is indexed by five values $1,2,\ldots,N$. The probability matrix is described
in the tutorial referenced above, but for $N=5$ it takes the following form
$$
\begin{pmatrix}
 0.437956  & 0.218978 & 0.145985 & 0.109489  & 0.0875912\\
 0.193548  & 0.387097 & 0.193548 & 0.129032  & 0.0967742\\
 0.125000  & 0.187500 & 0.375000 & 0.187500  & 0.1250000\\
 0.0967742 & 0.129032 & 0.193548 & 0.387097  & 0.1935480\\
 0.0875912 & 0.109489 & 0.145985 & 0.218978  & 0.4379560\\
\end{pmatrix}.
$$
The incumbency status changes according to the action last period. We normalize
the utility of a firm when it is not operating in the market to zero. If the firm
is operating, the utility (or profit) is
$$
U(enter|M_t, a_{t-1}) = (1; M_t; a_{t-1}-1)'\theta,
$$
where $\theta$ is a parameter vector we want to estimate. The first term is a
fixed cost of operating, the second is the market variable, and the last term is
the entry cost. For simulation purposes it is set to $(-.01; .20; 1.00)$,
and the discount factor is $0.99$.

This model is very similar to the model above, with small differences. The demand
specification is linear in the state, but more importantly the transition matrix is dense.

First, we construct the states

In [1]:
using StocasEstimators

In [2]:
# Incumbancy state
X1 = ["not present", "present"]
F1 = [[1 0; 1 0], [0 1; 0 1]]
S1 = State("incumbancy", X1, F1)
# Market demand state
X2 = collect(1:5)
nX2 = length(X2)
F2 = 1./(1+abs.(ones(nX2,1)*X2'-X2*ones(1, nX2)))
F2 = F2./sum(F2,1)'
S2 = CommonState("market demand", X2, F2)
# States
S = States(S1, S2)

Discrete states
 * Number of state variables: 2


Notice how we constructed the entry state the exact same way as before, although dense this time.
Since such a state is often used in these types of models, there's a handy \texttt{EntryState()}
constructor that can be used

In [3]:
S_entry = EntryState() # a dense representation
# S_entry_sparse = EntryState(dense = false) # a sparse representation

 * Total number of states:    10
 * Individual states:           
   1) incumbancy (n: 2)
   2) market demand (n: 5)
Discrete state


Then we need to construct our payoff type instance

In [4]:
# Payoff variables
Z1 = zeros(nX2*2, 3)
Z2 = [ones(nX2) X2 -ones(nX2);
      ones(nX2) X2 zeros(nX2)]
U = LinearUtility(("don't enter", "enter"), (Z1, Z2), 0.95, [-.1;.2;1])

 * name: incumbancy
 * n: 2


Stocas.LinearUtility{2,Float64}(("don't enter", "enter"), ([0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0], [1.0 1.0 -1.0; 1.0 2.0 -1.0; … ; 1.0 4.0 0.0; 1.0 5.0 0.0]), 0.95, [-0.1, 0.2, 1.0], ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-0.9, -0.7, -0.5, -0.3, -0.1, 0.1, 0.3, 0.5, 0.7, 0.9]), ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]), 3)

This completes the model specification.

We solve the model as above. Benchmarking again shows that Newton and policy iterations are very close, and this
time only slightly faster than VFIs. Since $\beta$ is only $0.95$ this time around, it
only takes 464 iterations to converge if we start from a $V=0$ guess. The two other
methods are still faster though, and this is related to the fact that the state space
only contains 10 possible states compared to 175 in the bus engine example.

In [5]:
Vvfi, itersvfi = solve(U, S, Stocas.VFI())
Vnewt, itersnewt = solve(U, S, Stocas.Newton())
Vpoli, iterspoli = solve(U, S, Stocas.Policy())
Vpoly, iterspoly = solve(U, S, Stocas.Poly())

(Stocas.IntegratedValueFunction{Array{Float64,1},Array{Float64,2},Array{Array{Float64,1},1}}([14.8609, 15.015, 15.2073, 15.4157, 15.6145, 15.403, 15.6093, 15.8527, 16.1091, 16.3509], [14.8609, 15.015, 15.2073, 15.4157, 15.6145, 15.403, 15.6093, 15.8527, 16.1091, 16.3509], [0.241837 0.120919 … 0.0435553 0.0348442; 0.097006 0.194012 … 0.05791 0.0434325; … ; 0.0192029 0.0256038 … 0.29093 0.145465; 0.0146046 0.0182557 … 0.171518 0.343035], Array{Float64,1}[[14.3184, 14.3755, 14.457, 14.543, 14.6109, 14.3184, 14.3755, 14.457, 14.543, 14.6109], [14.8904, 14.9651, 15.0681, 15.1748, 15.258, 14.8904, 14.9651, 15.0681, 15.1748, 15.258]]), [12, 2])

We simulate 20 firms for 15 years.

In [6]:
M = 5
N = 20
T = 12*15
D = simulate(U, S, M, N, T)

Stocas.Data{Int64}([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20, 20, 20, 20, 20, 20, 20, 20, 20, 20], [1, 2, 2, 2, 2, 2, 2, 1, 1, 1  …  2, 1, 2, 1, 2, 2, 2, 1, 2, 2], Array{Int64,1}[[1, 8, 9, 10, 11, 12, 29, 30, 32, 47  …  17955, 17964, 17965, 17966, 17979, 17980, 17981, 17992, 17994, 17998], [2, 3, 4, 5, 6, 7, 13, 14, 15, 16  …  17988, 17989, 17990, 17991, 17993, 17995, 17996, 17997, 17999, 18000]], [1, 3, 8, 6, 10, 10, 10, 7, 1, 3  …  8, 7, 2, 8, 5, 10, 9, 9, 4, 9], [1 1; 1 3; … ; 1 4; 2 4], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  5, 5, 5, 5, 5, 5, 5, 5, 5, 5], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [755 589; 802 657; … ; 411 1687; 439 1987], 18000)

All that is left to do now, is simply to estimate the market transitions, construct the data type, and estimate using both NFXP and NPL.

In [7]:
# States based on estimated transitions
# Estimate market probability matrix
F2_est = zeros(5, 5)
for n = 1:T*N
   if n%T != 0
      F2_est[D.xs[n,2], D.xs[n + 1,2]] += 1
   end
end

F2_est = F2_est./vec(sum(F2_est,2))
S_est = States(S1, CommonState(X2, F2_est))
display(S_est)

Discrete states
 * Number of state variables: 2
 * Total number of states:    10
 * Individual states:           
   1) incumbancy (n: 2)
   2) common state (n: 5)


And now we estimate using NFXP and NPL.

In [8]:
nfxp_result = fit_nfxp(U, S_est, D)
npl_result = fit_npl(U, S_est, D)
display(nfxp_result), println(), display(npl_result);

Results of estimation
 * Method: NFXP
 * loglikelihood: -10701.27439256443


 * Estimate: [-0.050657422434448166,0.18100022193275397, ...]
 * Std.err.: [0.03329392931024208,0.011008337958963048, ...]
 Iterations
 * Maximum likelihood: 3
 * Contractions: 73
 * Newton steps: 10

Results of estimation


 * Method: NPL
 * loglikelihood: -10701.274392565932
 * Estimate: [-0.05065739086026976,0.18100020840640463,1.076090599402945]
 * Std.err.: [0.0332281473914808,0.011000350624813979,0.03373500270931194]
 Iterations
 * Policy iteration: 4
 * Maximum likelihood: 4


We see that the estimates are identical.

In [9]:
using Plots
pgfplots()
plot(("not present", :), S, U, :policy, title="Entry probabilities for different market demand.")

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 Z M 2.9375 -6.375 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.125 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 L 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 Z M 1.265625 -0.765625 "/>
 
 
<path style="stroke:none;" d="M 2.890625 -3.515625 C 3.703125 -3.78125 4.28125 -4.46875 4.28125 -5.265625 C 4.28125 -6.078125 3.40625 -6.640625 2.453125 -6.640625 C 1.453125 -6.640625 0.6875 -6.046875 0.6875 -5.28125 C 0.6875 -4.953125 0.90625 -4.765625 1.203125 -4.765625 C 1.5 -4.765625 1.703125 -4.984375 1.703125 -5.28125 C 1.703125 -5.765625 1.234375 -5.765625 1.09375 -5.765625 C 1.390625 -6.265625 2.046875 -6.390625 2.40625 -6.390625 C 2.828125 -6.390625 3.375 -6.171875 3.375 -5.28125 C 3.375 -5.15625 3.34375 -4.578125 3.09375 -4.140625 C 2.796875 -3.65625 2.453125 -3.625 2.203125 -3.625 C 2.125 -3.609375 1.890625 -3.59375 1.8125 -3.59375 C 1.734375 -3.578125 1.671875 -3.5625 1.671875 -3.46875 C 1.671875 -3.359375 1.734375 -3.359375 1.90625 -3.359375 L 2.34375 -3.359375 C 3.15625 -3.359375 3.53125 -2.6875 3.53125 -1.703125 C 3.53125 -0.34375 2.84375 -0.0625 2.40625 -0.0625 C 1.96875 -0.0625 1.21875 -0.234375 0.875 -0.8125 C 1.21875 -0.765625 1.53125 -0.984375 1.53125 -1.359375 C 1.53125 -1.71875 1.265625 -1.921875 0.984375 -1.921875 C 0.734375 -1.921875 0.421875 -1.78125 0.421875 -1.34375 C 0.421875 -0.4375 1.34375 0.21875 2.4375 0.21875 C 3.65625 0.21875 4.5625 -0.6875 4.5625 -1.703125 C 4.5625 -2.515625 3.921875 -3.296875 2.890625 -3.515625 Z M